# Sparklehorse - Der SQL-Chatbot der Magpie

## Vorbereitung der Arbeitsumgebung

In einem ersten Schritt definierne wir unser Arbeitsverzeichnis. 

In [1]:
import os
os.getcwd()
os.chdir("c:/Users/mhu/Documents/gitHub/magpie_chatbot")
# Pfad Privatrechner
# os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/magpie_langchain")

Lade Umgebungsvariablen (inkl. OpenAI-API-Key) und initialisiere den Chatbot mit dem Modell "gpt-4o" von OpenAI.

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
llm = ChatOpenAI(model="gpt-4o")

  Im folgenden stellen wir Verbindung zur Magpie her. Wir schauen uns dan alle in der Magpie befindlichen Tabellen an. Schließlich wählen wir `view_daten_reichweite_menge` aus und speichern diese als Pandas Data Frame zur einfachen exploration der Datentabelle.

In [3]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit 

db = SQLDatabase.from_uri("duckdb:///data/view_magpie.db") 

db.run("SHOW TABLES")

query = "SELECT * FROM view_daten_reichweite_menge;"
df = pd.read_sql(query, db._engine)
df

c:\Users\mhu\AppData\Local\miniconda3\envs\chatbot_magpie\lib\site-packages\duckdb_engine\__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


,daten_id,variable_id,variable_beschr,zeit_start,zeit_ende,zeit_einheit,wert,wert_einheit,reichweite_az,reichweite_id_list,reichweite_beschr_list,reichweite_id_beschr_list,reichweite_typ_list,typ_list,reichweite_klasse_list,klasse_list,reichweite_menge_id_list,menge_zugeh_reichweiten,quelle_list,tag_list
0,1,1,Interne FuE-Aufwendungen,1983-01-01,1983-12-31 23:59:59,Jahr,21809155,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Stifterverband | SV Wissenschaftsstatistik |...,| Forschung und Entwicklung | --- | Wirtschaft |
1,2,1,Interne FuE-Aufwendungen,1985-01-01,1985-12-31 23:59:59,Jahr,25629000,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Stifterverband | SV Wissenschaftsstatistik |...,| Forschung und Entwicklung | --- | Wirtschaft |
2,3,1,Interne FuE-Aufwendungen,1987-01-01,1987-12-31 23:59:59,Jahr,29212150,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Destatis | --- | Stifterverband | SV Wissens...,| Forschung und Entwicklung | --- | Wirtschaft |
3,4,1,Interne FuE-Aufwendungen,1989-01-01,1989-12-31 23:59:59,Jahr,32577985,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Stifterverband | SV Wissenschaftsstatistik |...,| Forschung und Entwicklung | FuE-Personal | -...
4,5,1,Interne FuE-Aufwendungen,1993-01-01,1993-12-31 23:59:59,Jahr,38624000,in Tsd. Euro,4.940656e-324,10,Deutschland,[10] Deutschland,Deutschland::Land,Land,Deutschland::Räumliche Gebiete,Räumliche Gebiete,None,None,| Destatis | --- | Stifterverband | SV Wissens...,| Forschung und Entwicklung | FuE-Personal | -...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264779,288651,231,Studierende,2022-10-01,2023-03-31 23:59:59,Wintersemester,32,Anzahl,1.482197e-323,"5,735,1001",weiblich | Nicht deutsch | Zoll- Steuerverwaltung,[5] weiblich | [735] Nicht deutsch | [1001] Zo...,weiblich::Geschlecht | Nicht deutsch::Staatsan...,Geschlecht | Staatsangehörigkeit | Studienfach,weiblich::soziodemographische Merkmale | Nicht...,Hochschulbildung | soziodemographische Merkmal...,5::3,"weiblich, männlich",| Destatis |,None
264780,288652,231,Studierende,2022-10-01,2023-03-31 23:59:59,Wintersemester,47,Anzahl,9.881313e-324,"735,1001",Nicht deutsch | Zoll- Steuerverwaltung,[735] Nicht deutsch | [1001] Zoll- Steuerverwa...,Nicht deutsch::Staatsangehörigkeit | Zoll- Ste...,Staatsangehörigkeit | Studienfach,Nicht deutsch::soziodemographische Merkmale | ...,Hochschulbildung | soziodemographische Merkmale,None,None,| Destatis |,None
264781,288653,231,Studierende,2022-10-01,2023-03-31 23:59:59,Wintersemester,3363,Anzahl,9.881313e-324,"6,1001",männlich | Zoll- Steuerverwaltung,[6] männlich | [1001] Zoll- Steuerverwaltung,männlich::Geschlecht | Zoll- Steuerverwaltung:...,Geschlecht | Studienfach,männlich::soziodemographische Merkmale | Zoll-...,Hochschulbildung | soziodemographische Merkmale,6::3,"weiblich, männlich",| Destatis |,None
264782,288654,231,Studierende,2022-10-01,2023-03-31 23:59:59,Wintersemester,4022,Anzahl,9.881313e-324,"5,1001",weiblich | Zoll- Steuerverwaltung,[5] weiblich | [1001] Zoll- Steuerverwaltung,weiblich::Geschlecht | Zoll- Steuerverwaltung:...,Geschlecht | Studienfach,weiblich::soziodemographische Merkmale | Zoll-...,Hochschulbildung | soziodemographische Merkmale,5::3,"weiblich, männlich",| Destatis |,None


## Tools

### Standardisierte Langchain Tools

Wir intitalsieren ein standardisiertes Toolkit. Es stellt Funktionen bereit, um SQL-Queries über natürliche Sprache zu erzeugen und auszuführen. Wir lassen uns Namen und Funktion der standardisierten Tools anzeigen:

In [4]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {tool.description}")
    print("-" * 40)

Tool Name: sql_db_query
Description: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
----------------------------------------
Tool Name: sql_db_schema
Description: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
----------------------------------------
Tool Name: sql_db_list_tables
Description: Input is an empty string, output is a comma-separated list of tables in the database.
----------------------------------------
Tool Name: sql_db_query_checker
Description: Use this tool to double check if your query is

Die Standardtools aus`SQLDatabaseToolkit`können also die folgenden Standardfunktionen übernehmen: 

- `sql_db_query`  
  Führt eine übergebene SQL-Abfrage aus. Gibt das Ergebnis oder eine Fehlermeldung zurück. Bei Fehlern wie „Unknown column“ sollte zuvor das Tabellenschema geprüft werden.

- `sql_db_schema`  
  Gibt das Schema (Spaltennamen und -typen) sowie Beispielzeilen für angegebene Tabellen zurück. Vorher sollte geprüft werden, ob die Tabellen existieren.

- `sql_db_list_tables`  
  Listet alle Tabellen in der verbundenen Datenbank auf.

- `sql_db_query_checker`  
  Prüft eine SQL-Abfrage auf syntaktische Korrektheit, bevor sie mit sql_db_query ausgeführt wird. Sollte immer vorher verwendet werden.


### Maßgeschneiderte Langchain Tools

#### Retriever `rt_beschr_variable`

`rt_beschr_variable` erlaubt die semantischen Suche über Werte aus einer Datenbankspalte. 

1. Wir sammeln sämtliche Unique Werte aus `beschr_variable` und wandeln diese mit OpenAIs Embeddings-Methdode `text-embedding-3-large` in Embeddings um. Die werden in einen Vektorstore gesichert.
2. Der Vektorstore wird in einen Retriever umgewandelt, der bei einer Anfrage die 5 ähnlichsten Begriffe zurückgibt.
3. Schließlich wird mit `create_retriever_tool` ein Tool erzeugt, das den Retriever kapselt. Dieses Tool kann von Sparklehorse genutzt werden, um Benutzereingaben mit unsicherer Schreibweise oder unvollständigen Begriffen mit den tatsächlichen Werten in der Datenbank abzugleichen.

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents.agent_toolkits import create_retriever_tool
import ast
import re

##################################################################
# Generiere `rt_beschr_variable`
##################################################################

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


beschr_variable = query_as_list(db, "SELECT variable_beschr FROM view_daten_reichweite_menge")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_texts(beschr_variable)

retriever_beschr_variable  = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Verwenden, um Werte für Filterabfragen nachzuschlagen. Die Eingabe ist eine ungefähre Schreibweise "
    "eines Eigennamens, die Ausgabe sind gültige Eigennamen. Verwende den Begriff, der der Eingabe am ähnlichsten ist."
)

rt_beschr_variable = create_retriever_tool(
    retriever_beschr_variable,
    name="rt_beschr_variable",
    description=description,
)

Wir testen nun den Retriever:

In [6]:
def print_clean_result(result):
    print("\n".join(result.split("\n\n")))

result = rt_beschr_variable.invoke("Studienanfänger ohne Abitur")

print_clean_result(result)

Studienabsolventen ohne Abitur
Anzahl der Studienanfänger ohne Abitur
Anteil der Studienanfänger ohne Abitur
Anteil der Studienabsolventen ohne Abitur
Studienabsolventen Bildungsausländer


#### Retriever `rt_reichweite_variable`

Wir bauen einen baugleichen Retriever nun noch für die Variablen `reichweite_beschr_list` und `wert_einheit`. 

In [7]:
# ##################################################################
# # Generiere `rt_reichweite_variable`
# ##################################################################

# reichweite_variable = query_as_list(db, "SELECT reichweite_beschr_list FROM view_daten_reichweite_menge")

# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# vector_store = InMemoryVectorStore(embeddings)

# _ = vector_store.add_texts(reichweite_variable)

# retriever_reichweite_variable = vector_store.as_retriever(search_kwargs={"k": 5})

# description = (
# "Verwenden, um Werte für Filterabfragen nachzuschlagen. Die Eingabe ist eine ungefähre Schreibweise "
# "eines Eigennamens, die Ausgabe sind gültige Eigennamen. Verwende den Begriff, der der Suche am ähnlichsten ist."
# )

# rt_reichweite_variable = create_retriever_tool(
#     retriever_reichweite_variable,
#     name="rt_reichweite_variable",
#     description=description,
# )

In [8]:
# result = rt_reichweite_variable.invoke("Wie hoch war die Anzahl von Studierenden ohne Abitur 2006?")

# print_clean_result(result)

In [9]:
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

@tool
def variable_beschr(user_question: str) -> str:
    """
    Nutzt ein LLM, um aus der Frage eine passende Variable zu extrahieren,
    und gibt dann die exakte Variable aus der Datenbank zurück.
    """
    extract_prompt = PromptTemplate(
        input_variables=["frage"],
        template="""
        Die folgende Frage bezieht sich auf eine statistische Kennzahl.
        Extrahiere bitte eine passende Variable, die in einer Datenbankabfrage verwendet werden kann. Extrahiere ebenfalls, wenn eine Anzahl, oder Anteil gewünscht ist.

        Beispielausgabe: 'Anteil Studienanfänger ohne Abitur', 'Drittmittel DFG', 'Technisches FuE-Personal'

        Frage: "{frage}"

        Gib nur eine Variable zurück.
        """
    )
    variable_chain = extract_prompt | llm
    extracted_variable = variable_chain.invoke({"frage": user_question}).content.strip()

    docs = retriever_beschr_variable.get_relevant_documents(extracted_variable)
    if not docs:
        return "Error: Keine passende Variable gefunden."

    kandidaten = "\n".join(f"- {doc.page_content.strip()}" for doc in docs)

    auswahl_prompt = PromptTemplate(
        input_variables=["frage", "kandidaten"],
        template="""
        Wähle aus den folgenden Variablen diejenige, die am besten zur Frage passt.
        
        Frage: "{frage}"

        Kandidaten:
        {kandidaten}

        Gib **nur** die beste passende Variable exakt zurück.
        """
    )
    auswahl_chain = auswahl_prompt | llm
    best_match = auswahl_chain.invoke({
        "frage": user_question,
        "kandidaten": kandidaten
    }).content.strip()

    query = f"""
        SELECT variable_beschr 
        FROM view_daten_reichweite_menge 
        WHERE variable_beschr = '{best_match}' 
        LIMIT 1;
    """
    result = db.run_no_throw(query)
    return result if result else "Error: Keine passende Variable gefunden."


@tool
def get_reichweite_beschr_list(user_question: str) -> str:
    """
    Ermittelt eine passende Reichweite (z. B. Region, Organisation, etc.), aber nur aus denjenigen,
    die zur durch variable_beschr extrahierten Variable in der Datenbank gehören.
    """
    raw_variable = variable_beschr.run(user_question)

    variable = (
        str(raw_variable)
        .replace("[", "")
        .replace("]", "")
        .replace("(", "")
        .replace(")", "")
        .replace("'", "")
        .replace(",", "")
        .strip()
    )

    if "Error" in variable:
        return "Fehler: Konnte keine gültige Variable bestimmen."

    escaped_variable = variable.replace("'", "''")

    extract_prompt = PromptTemplate(
        input_variables=["frage"],
        template="""
        Extrahiere eine Reichweite wie Region, Hochschule, Geschlecht o. ä. Sekotoren
        Wenn nichts genannt wird, gib 'Deutschland' zurück.

        Frage: "{frage}"

        Antwort:
        """
    )
    reichweite_chain = extract_prompt | llm
    reichweite = reichweite_chain.invoke({"frage": user_question}).content.strip()

    query = f"""
        SELECT DISTINCT reichweite_beschr_list 
        FROM view_daten_reichweite_menge 
        WHERE variable_beschr = '{escaped_variable}'
    """
    gültige_reichweiten = query_as_list(db, query)
    if not gültige_reichweiten:
        return "Fehler: Keine passenden Reichweiten gefunden."

    vector_store = InMemoryVectorStore(OpenAIEmbeddings(model="text-embedding-3-large"))
    _ = vector_store.add_texts(gültige_reichweiten)
    retriever_reichweite_variable = vector_store.as_retriever(search_kwargs={"k": 5})

    description = (
        "Verwenden, um Werte für Filterabfragen nachzuschlagen. Die Eingabe ist eine ungefähre Schreibweise "
        "eines Eigennamens, die Ausgabe sind gültige Eigennamen. Verwende den Begriff, der der Suche am ähnlichsten ist."
    )

    rt_reichweite_variable = create_retriever_tool(
        retriever_reichweite_variable,
        name="retriever_reichweite_variable",
        description=description,
    )
    matches = rt_reichweite_variable.invoke(reichweite).split("\n")
    best_match = matches[0].strip()

    query = f"""
        SELECT reichweite_beschr_list 
        FROM view_daten_reichweite_menge 
        WHERE reichweite_beschr_list = '{best_match}' 
        LIMIT 1;
    """
    result = db.run_no_throw(query)
    return result if result else "Error: Keine passende Variable gefunden."

tools.extend([variable_beschr, get_reichweite_beschr_list])

In [10]:
test_input = "Wie viele Studienanfänger ohne Abitur gab es 2019 im Saarland?"
output = variable_beschr(test_input)
print(output)

C:\Users\mhu\AppData\Local\Temp\ipykernel_79360\1539016302.py:2: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = variable_beschr(test_input)
C:\Users\mhu\AppData\Local\Temp\ipykernel_79360\23383328.py:27: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever_beschr_variable.get_relevant_documents(extracted_variable)


[('Anzahl der Studienanfänger ohne Abitur',)]


In [11]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1, "Die Anzahl der Nachrichten im Template ist nicht 1!"
# Bearbeite die bestehende Nachricht, indem du Text hinzufügst
prompt_template.messages[0].prompt.template += (
    "\nYou are Sparklehorse, a chatbot for the Stifterverband organization. "
    "Your primary task is to answer questions related to the Magpie database."
)

prompt_template.messages[0].pretty_print()

c:\Users\mhu\AppData\Local\miniconda3\envs\chatbot_magpie\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [12]:
system_message = prompt_template.format(
    dialect=db.dialect, 
    top_k=5
)

print(system_message)

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct duckdb query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see w

In [13]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Systemnachricht mit extra Anweisungen
suffix = (
    "Bevor du eine SQL-Abfrage generierst, beachte bitte folgende Regeln strikt:\n"
    "1. Nutze das Tool `variable_beschr`, um die korrekte Variable aus der Nutzerfrage zu bestimmen. Verwende ausschließlich den exakten Rückgabewert dieses Tools für `variable_beschr` in der SQL-Abfrage.\n"
    "2. Nutze das Tool `get_reichweite_beschr_list`, um die passende Reichweite zu ermitteln. Verwende ausschließlich den Rückgabewert dieses Tools für `reichweite_beschr_list` in der SQL-Abfrage.\n"
    "3. Verwende **niemals** andere Felder wie `tag_list` oder `LIKE`-Abfragen. Nutze **immer exakte Vergleiche** mit `=`.\n"
    "4. Verwende ausschließlich die Tabelle `view_daten_reichweite_menge` für alle Abfragen.\n"
    "5. Falls ein Jahr in der Frage genannt wird, filtere mit `date_part('year', zeit_start) = <Jahr>`.\n"
    "6. Berücksichtige die Spalte `wert_einheit`, z. B. 'in Tsd. Euro', 'Anzahl', 'Prozent', 'VZÄ', 'Mitarbeiter'.\n"
    "7. Gib immer die finale SQL-Abfrage vollständig aus und erkläre sie. Rate niemals IDs oder Werte.\n"
    "8. Falls keine passende Variable oder Reichweite gefunden wurde, rate nicht irgendwelche Werte. \n"
    "9. Stelle sicher, dass Antworttext und SQL-Abfrage immer auf den gleichen `variable_beschr`- und `reichweite_beschr_list`-Werten basieren, um Konsistenz zu gewährleisten."
)

 

system = f"{system_message}\n\n{suffix}"

# Neuen ReAct-Agent erstellen mit den vollständigen Tools
agent_executor = create_react_agent(llm, tools, state_modifier=system)


In [14]:
# Testanfrage an den Agenten
question = "Wie viele Mitarbeitende wurden 2021 im Bereich technisches fue-personal gezählt?"

for step in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]}, 
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Wie viele Mitarbeitende wurden 2021 im Bereich technisches fue-personal gezählt?
================================== Ai Message ==================================
Tool Calls:
  variable_beschr (call_qCDnkuRZoGg5z2LjQLI8ZzZS)
 Call ID: call_qCDnkuRZoGg5z2LjQLI8ZzZS
  Args:
    user_question: Wie viele Mitarbeitende wurden 2021 im Bereich technisches fue-personal gezählt?
  get_reichweite_beschr_list (call_elwS7ouQjmEohnNK8tIt4rdO)
 Call ID: call_elwS7ouQjmEohnNK8tIt4rdO
  Args:
    user_question: Wie viele Mitarbeitende wurden 2021 im Bereich technisches fue-personal gezählt?
================================= Tool Message =================================
Name: get_reichweite_beschr_list

[('Wirtschaftssektor | Deutschland',)]
================================== Ai Message ==================================
Tool Calls:
  sql_db_query_checker (call_DPtX0EYo8NWYdD2okV9BGTNX)
 Call ID: call_DPtX0EYo8NWYdD2okV9B

In [16]:
import pandas as pd
from langchain.schema import HumanMessage

df = pd.read_excel("data/test_quest_mixed_nr_1.xlsx")
for question in df["Frage"].dropna():
    result = agent_executor.invoke({"messages": [HumanMessage(content=question)]})
    messages = result["messages"]
    antwort = messages[-1].content
    print(f"\nFrage: {question}\nAntwort: {antwort}")



Frage: Wie hoch waren die Ausgaben im Bereich Drittmittel vom Bund 2019?
Antwort: Die Ausgaben im Bereich "Drittmittel vom Bund" für das Jahr 2019 in Deutschland betrugen 2.527.227,037 Tausend Euro.

Frage: Wie hoch waren die Ausgaben im Bereich Drittmittel von der Bundesanstalt für Arbeit 2019?
Antwort: Es wurde keine passende Variable für die Anfrage "Wie hoch waren die Ausgaben im Bereich Drittmittel von der Bundesanstalt für Arbeit 2019?" gefunden. 

Falls die Anfrage missverständlich war oder zusätzliche Kontextinformationen fehlen, die zu einer erfolgreichen Suche führen könnten, lasse es mich bitte wissen oder stelle die Frage anders.

Frage: Wie hoch waren die Ausgaben im Bereich Drittmittel von der DFG 2019?
Antwort: Die Ausgaben im Bereich Drittmittel von der DFG im Jahr 2019 beliefen sich auf insgesamt 2.744.740,51 Tsd. Euro.

Frage: Wie hoch waren die Ausgaben im Bereich Drittmittel von der EU 2019?
Antwort: Die Ausgaben im Bereich "Drittmittel von der EU" in Deutschland i

KeyboardInterrupt: 

In [ ]:
from langgraph.prebuilt import create_react_agent
graph = create_react_agent(llm, tools, state_modifier=system)


In [ ]:
graph